# LangChain: Memory

## Outline
* RunnableWithMessageHistory
  * Analogous to using ConversationChain (deprecated) with the default ConversationBufferMemory
  * Make llm can answer question based on memory
* Simulate ConversationBufferWindowMemory to be used by RunnableWithMessageHistory
  * The first solution: Keep only last k chats in memory
  * The second solution: Just pass the last k chats from the memory to the llm, the memory will not be affected, as previous data not affected and keep adding new data to it.
* ConversationTokenBufferMemory
  * Not consider integrate it with RunnableWithMessageHistory until we have good usecase or example of using it
* Simulate ConversationSummaryMemory to be used by RunnableWithMessageHistory

## Reference
* [How to add memory to chatbots](https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/)

## RunnableWithMessageHistory

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4o-mini"

In [3]:
%pip install -U langchain-openai
%pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in memory:
        memory[session_id] = InMemoryChatMessageHistory()
    return memory[session_id]


with_message_history = RunnableWithMessageHistory(llm, get_session_history)
config = {"configurable": {"session_id": "InMemoryChatMessageHistory"}}

In [5]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Joseph")],
    config=config,
)

response.content

'Hi Joseph! How can I assist you today?'

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="What is 1+1?")],
    config=config,
)
response.content

'1 + 1 equals 2.'

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="What is ny name?")],
    config=config,
)
response.content

'Your name is Joseph.'

In [8]:
print(memory.get(config.get("configurable").get("session_id")))

Human: Hi, my name is Joseph
AI: Hi Joseph! How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2.
Human: What is ny name?
AI: Your name is Joseph.


## ConversationBufferWindowMemory

Solution 1: Simulating ConversionBufferWindowMemory()

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from typing import List

class ConversationBufferWindowMemory(BaseChatMessageHistory):
    """A chat message history that only keeps the last K messages."""
    
    def __init__(self, buffer_size: int):
        super().__init__()
        self.buffer_size = buffer_size
        self.messages: List[BaseMessage] = []

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store, keeping only the last K."""
        self.messages.extend(messages)
        # Ensure only the last K messages are kept
        self.messages = self.messages[-self.buffer_size:]

    def clear(self) -> None:
        """Clear the chat message history."""
        self.messages = []

# Example usage with RunnableWithMessageHistory
def get_conversation_buffer(buffer_size):
    def internal(session_id):
    # Assuming each session has its own buffer size requirement, 
    # you could customize this. For simplicity, we'll use a fixed size.
      return ConversationBufferWindowMemory(buffer_size=buffer_size)
    return internal

# Assuming you have a Runnable `chain` already defined
chain_with_limited_history = RunnableWithMessageHistory(
    llm,
    get_session_history=get_conversation_buffer(1),
)
              

In [10]:
response = chain_with_limited_history.invoke(
    [HumanMessage(content="Hi, my name is Joseph")],
    config=config,
)
response.content

'Hi Joseph! How can I assist you today?'

In [11]:
response = chain_with_limited_history.invoke(
    [HumanMessage(content="What is 1+1?")],
    config=config,
)
response.content

'1 + 1 equals 2.'

In [12]:
response = chain_with_limited_history.invoke(
    [HumanMessage(content="What is ny name?")],
    config=config,
)
response.content

"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. If you'd like to tell me your name, feel free!"

Solution 2: Use trimming history data provided by official website

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("Hey there! I'm Joseph.")
demo_ephemeral_chat_history.add_ai_message("Hello!")
demo_ephemeral_chat_history.add_user_message("How are you today?")
demo_ephemeral_chat_history.add_ai_message("Fine thanks!")
demo_ephemeral_chat_history.add_user_message("My mom is Maggie mee.")
demo_ephemeral_chat_history.add_ai_message("Oh i see!")

demo_ephemeral_chat_history.messages

[HumanMessage(content="Hey there! I'm Joseph."),
 AIMessage(content='Hello!'),
 HumanMessage(content='How are you today?'),
 AIMessage(content='Fine thanks!'),
 HumanMessage(content='My mom is Maggie mee.'),
 AIMessage(content='Oh i see!')]

In [15]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chat = ChatOpenAI(temperature=0.0, model=llm_model)
chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

chain_with_message_history.invoke(
    {"input": "What's my name?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='Your name is Joseph.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 81, 'total_tokens': 86}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a212343-4b4e-472a-8104-f4740fbe05bc-0', usage_metadata={'input_tokens': 81, 'output_tokens': 5, 'total_tokens': 86})

In [21]:
from operator import itemgetter

from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough

# The demo_ephemeral_chat_history will not be affected by the trimming
# This just get the last 2 messages from the demo_ephemeral_chat_history
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt
    | chat
)

chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: demo_ephemeral_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [22]:
response = chain_with_trimmed_history.invoke(
    {"input": "Where does P. Sherman live?"},
    {"configurable": {"session_id": "unused"}},
)
response.content

'P. Sherman lives at 42 Wallaby Way, Sydney. This address is a reference from the animated movie "Finding Nemo."'

In [23]:
demo_ephemeral_chat_history.messages

[HumanMessage(content="Hey there! I'm Joseph."),
 AIMessage(content='Hello!'),
 HumanMessage(content='How are you today?'),
 AIMessage(content='Fine thanks!'),
 HumanMessage(content='My mom is Maggie mee.'),
 AIMessage(content='Oh i see!'),
 HumanMessage(content="What's my name?"),
 AIMessage(content='Your name is Joseph.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 81, 'total_tokens': 86}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a212343-4b4e-472a-8104-f4740fbe05bc-0', usage_metadata={'input_tokens': 81, 'output_tokens': 5, 'total_tokens': 86}),
 HumanMessage(content='Where does P. Sherman live?'),
 AIMessage(content='P. Sherman lives at 42 Wallaby Way, Sydney. This is a reference from the animated movie "Finding Nemo."', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 101, 'total_tokens': 126}, 'model_name': 'gpt-4o-mini-2024-07-18',

In [24]:
response = chain_with_trimmed_history.invoke(
    {"input": "What is my mother name?"},
    {"configurable": {"session_id": "unused"}},
)
response.content

"I'm sorry, but I don't have access to personal information about individuals, including your mother's name. If you have any other questions or need assistance with something else, feel free to ask!"

In [25]:
demo_ephemeral_chat_history.messages

[HumanMessage(content="Hey there! I'm Joseph."),
 AIMessage(content='Hello!'),
 HumanMessage(content='How are you today?'),
 AIMessage(content='Fine thanks!'),
 HumanMessage(content='My mom is Maggie mee.'),
 AIMessage(content='Oh i see!'),
 HumanMessage(content="What's my name?"),
 AIMessage(content='Your name is Joseph.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 81, 'total_tokens': 86}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ba606877f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a212343-4b4e-472a-8104-f4740fbe05bc-0', usage_metadata={'input_tokens': 81, 'output_tokens': 5, 'total_tokens': 86}),
 HumanMessage(content='Where does P. Sherman live?'),
 AIMessage(content='P. Sherman lives at 42 Wallaby Way, Sydney. This is a reference from the animated movie "Finding Nemo."', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 101, 'total_tokens': 126}, 'model_name': 'gpt-4o-mini-2024-07-18',

## ConversationTokenBufferMemory

In [26]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [27]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [28]:
memory.load_memory_variables({})

{'history': 'AI: Charming!'}

## ConversationSummaryMemory

In [29]:
from langchain_community.chat_message_histories import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("Hey there! I'm Joseph.")
demo_ephemeral_chat_history.add_ai_message("Hello!")
demo_ephemeral_chat_history.add_user_message("How are you today?")
demo_ephemeral_chat_history.add_ai_message("Fine thanks!")
demo_ephemeral_chat_history.add_user_message("My mom is Maggie mee.")
demo_ephemeral_chat_history.add_ai_message("Oh i see!")

demo_ephemeral_chat_history.messages

[HumanMessage(content="Hey there! I'm Joseph."),
 AIMessage(content='Hello!'),
 HumanMessage(content='How are you today?'),
 AIMessage(content='Fine thanks!'),
 HumanMessage(content='My mom is Maggie mee.'),
 AIMessage(content='Oh i see!')]

In [30]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

chat = ChatOpenAI(temperature=0.0, model=llm_model)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [31]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = demo_ephemeral_chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    demo_ephemeral_chat_history.clear()

    demo_ephemeral_chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [32]:
response = chain_with_summarization.invoke(
    {"input": "What did I say my name was?"},
    {"configurable": {"session_id": "unused"}},
)
response.content

'You mentioned that your name is Joseph.'

In [33]:
print(demo_ephemeral_chat_history)

AI: Joseph introduced himself and mentioned that his mom is named Maggie Mee. He also asked how I was doing, to which I responded that I was fine.
Human: What did I say my name was?
AI: You mentioned that your name is Joseph.


In [34]:
response = chain_with_summarization.invoke(
    {"input": "What is my mother name?"},
    {"configurable": {"session_id": "unused"}},
)
response.content

"Your mother's name is Maggie Mee."

In [35]:
print(demo_ephemeral_chat_history)

AI: Joseph introduced himself and shared that his mom's name is Maggie Mee. He also inquired about my well-being, to which I responded that I was fine.
Human: What is my mother name?
AI: Your mother's name is Maggie Mee.
